<a href="https://colab.research.google.com/github/Gallicchio-Lab/AToM-OpenMM/blob/master/example-notebooks/atom_openmm_rbfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example of Protein-ligand Alchemical Relative Binding Free Energy Calculation with Atom-OpenMM

### Acknowledgements

- OpenMM: https://openmm.org
- openmmforcefields: https://github.com/openmm/openmmforcefields
- Espaloma force field: https://github.com/choderalab/espaloma
- Making-it-rain notebooks:  https://github.com/pablo-arantes/making-it-rain
- HPK1 system: https://doi.org/10.1021/acsmedchemlett.0c00672
- Thanks to Stefan Doerr @Acellera for helping modernizing the atom-openmm modules.

### Procedure

- In this example, the pdb file of a protein receptor (HPK1) and the sdf files of two ligands already docked to the receptor are combined into a complex.
- The second ligand is displaced into the solvent by a chosen displacement vector.
- The two ligands are aligned using 3 chosen corresponding reference atoms.
- The system is solvated, minimized, thermalized, and annealed at the alchemical intermediate (when the two ligands are half bound and half unbound).
- Then an alchemical Hamiltonian replica exchange simulation is carried out.
- Finally, the perturbation energy data collected during the simulation is analyzed to obtain the relative binding free energy of the two ligands.

Read the [introduction to AToM-OpenMM](https://www.compmolbiophysbc.org/atom-openmm) to get a better sense of steps involved.


### To run the notebook

- Connect to a Google Colab Runtime by clicking on the down-arrow next to the `Connect` button on the upper left, select `Change runtime type`, and selecting a `T4` GPU. Then click `Connect`.

- Execute the first cell to install `condacolab`. The runtime will restart. When done, move to the following cells.

- The final cells helps you download the simulation results.



### **Install Conda Colab (if on Google Colab)**

It will restart the kernel (it's okay, the warning is harmless)

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

### **Install dependencies**

In [ ]:
import subprocess
import sys

#fixes sys.path that gives torchvision import error
original_syspath = sys.path.copy()
new_syspath = ['/content', '/env/python', '/usr/local/lib/python312.zip', '/usr/local/lib/python3.12', '/usr/local/lib/python3.12/lib-dynload', '/usr/local/lib/python3.12/site-packages' ]
sys.path = new_syspath

#dependencies from conda-forge
subprocess.run("mamba install openmm openmmforcefields espaloma setproctitle nglview -y", shell=True)

#latest atom-openmm (or 'pip install atom-openmm' for the latest released version)
subprocess.run("cd /content && git clone https://github.com/Gallicchio-Lab/AToM-OpenMM.git", shell=True)
subprocess.run("cd /content/AToM-OpenMM && pip install .", shell=True)

### **Test the OpenMM installation**

In [ ]:
import openmm.testInstallation
openmm.testInstallation.main()

### **Prepare the ATM system in a box of water**

It requires the pdb file of the receptor and the sdf files of the ligands already docked to the receptor.

In this example, they are taken from the ATom-OpenMM repo. In general, you might want to upload them somewhere in the Colab instance.

It uses the basename of the receptor and ligand file names to contruct the job name (for example `hpk1-lig28-lig30`). The job working directory is taken as `/content/` followed by the job name.

In [ ]:
import os
from atom_openmm.make_atm_system_from_rcpt_lig import make_system
from pathlib import Path

#collect names of receptor pdb file and ligand sdf files
Protein_PDB_file_name = 'hpk1.pdb'
Ligand1_SDF_file_name = 'lig28.sdf'
Ligand2_SDF_file_name = 'lig30.sdf'

#where the receptor and ligand file name are stored
dataDir = '/content/AToM-OpenMM/example-notebooks/data'

#displacement vector
Displacement_vector_in_Angstroms = [0.0, 0.0, 45.0]

#full file names
rcpt_pdb = dataDir + '/' + Protein_PDB_file_name
ligand1_sdf = dataDir + '/' + Ligand1_SDF_file_name
ligand2_sdf = dataDir + '/' + Ligand2_SDF_file_name

#basename of the job (the job name)
basename = Path(Protein_PDB_file_name).stem + \
  '-' + \
  Path(Ligand1_SDF_file_name).stem + \
  '-' + \
  Path(Ligand2_SDF_file_name).stem \

#create the working directory and change to it
workDir = f'/content/{basename}'
os.makedirs(workDir, exist_ok=True)
os.chdir(workDir)

#the ATM system xml file and the output pdb file
outxml = workDir + '/' + basename + '_sys.xml'
pdboutfile = workDir + '/' + basename + '.pdb'

#builds the system
make_system(
      receptorfile=rcpt_pdb,
      lig1sdffile=ligand1_sdf ,
      lig2sdffile=ligand2_sdf,
      displacement = Displacement_vector_in_Angstroms,
      xmloutfile=outxml,
      pdboutfile=pdboutfile,
      ionicstrength=0.15,
      ligandforcefield='espaloma-0.3.2'
)


### **View the Protein-Ligand Complex in water**

The second ligand is displaced from the first in the solvent by the displacement vector.

Try a different displacement vector if this location is not satisfactory. For example, if it is too close to the receptor or, alternatively, if it leads to an excessively large simulation box.

In [ ]:

from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import nglview as nv
view = nv.show_structure_file(pdboutfile, default_representation = False)
view.add_cartoon('protein')
view.add_spacefill('ligand')
view.add_line('water')
view.add_ball_and_stick('ion')
view.layout.height = '800px'
view

### **Set the Ligand Alignment Atoms**

ATM required that the internal frames of the two ligands to be aligned. This is achieved by identifying three corresponding non-colinear atoms of each ligand. Usually, these are chosen from the rigid common core.

You can read more about alignment atoms [here](https://www.compmolbiophysbc.org/atom-openmm/atom-system-setup#h.vndnoxipr7qs).

Look at the structure of each ligand below and set the reference atoms using their ids in the `ligand1_ref_atoms` and `ligand2_ref_atoms` parameters. Or accept the reference atoms we picked for these specific ligands.


In [ ]:

#@title ##### **View Ligand 1 to Select Alignment Atoms, or Accept Defaults**
view = nv.show_structure_file(ligand1_sdf, default_representation = False)
view.add_line(selection='not hydrogen')
view.add_representation('label', selection='all', labelType="atomindex")
view

In [ ]:
view = nv.show_structure_file(ligand1_sdf, default_representation = False)
view.add_line(selection='not hydrogen')
ligand1_ref_atoms = [18, 14, 15] #@param
view.add_representation('label', selection=ligand1_ref_atoms, labelType="atomindex")
view

In [ ]:
#@title ##### **View Ligand 2 to Select Alignment Atoms, or Accept Defaults**

view = nv.show_structure_file(ligand2_sdf, default_representation = False)
view.add_line(selection='not hydrogen')
view.add_representation('label', selection='all', labelType="atomindex")
view

In [ ]:
view = nv.show_structure_file(ligand2_sdf, default_representation = False)
view.add_line(selection='not hydrogen')
ligand2_ref_atoms = [9, 1, 6] #@param
view.add_representation('label', selection=ligand2_ref_atoms, labelType="atomindex")
view

### **Setup Relative Binding Free Energy Calculation**

It does energy minimization, thermalization, equilibration, and annealing to the alchemical intermediate at lambda=1/2

It takes about 1/2 hour on a T4 GPU.

In [ ]:

import os
import openmm as mm
from openmm.app import *
from openmm import *
from openmm.unit import *
from multiprocessing import freeze_support
from atom_openmm.rbfe_structprep import rbfe_structprep
from atom_openmm.rbfe_production import rbfe_production

#get a list of atom indexes from a query
def get_indexes_from_query(topology, query):
    indexes = []
    for atom in topology.atoms():
        if eval(query):
            indexes.append(atom.index)
    indexes.sort()
    return indexes

#get the indexes of the atoms of a residue. Optionally, filter them by a query
def get_indexes_from_residue(residue, query = 'True'):
    indexes = []
    for atom in residue.atoms():
        if eval(query):
            indexes.append(atom.index)
    indexes.sort()
    return indexes

#calculates the center of a set of atoms
def cm_from_indexes(topology, positions, indexes):
    cm = Vec3(0,0,0)*nanometer
    n = 0
    for atom in topology.atoms():
        if atom.index in indexes:
            cm += positions[atom.index]
            n += 1
    cm = cm/float(n)
    return cm

options = {
        "JOB_TRANSPORT": 'LOCAL_OPENMM',
        "RE_SETUP": 'YES',
        "TEMPERATURES": [ 300.0 ],
        "LAMBDAS":      [0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00],
        "DIRECTION":    [   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
        "INTERMEDIATE": [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        "LAMBDA1":      [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        "LAMBDA2":      [0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00],
        "ALPHA":        [0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10],
        "U0":           [110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110.],
        "W0COEFF":      [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        "WALL_TIME": 240, #simulation time in minutes
        "CYCLE_TIME": 10, #interval between replica-exchange exchanges in seconds
        "CHECKPOINT_TIME": 1200, #interval between checkpoints in seconds
        "SUBJOBS_BUFFER_SIZE": 1.0, #fraction of replicas to keep in fast execution queue, in units of number of GPU devices
        "PRODUCTION_STEPS": 10000, #how many MD steps to run each replica
        "PRNT_FREQUENCY":   10000, #interval between printing replica data, in MD steps
        "TRJ_FREQUENCY":    10000, #interval between trajectory frame records, in MD steps
        "CM_KF": 25.00, #force constant of flat-bottom binding site restraint, in kcal/mol/Ang^2
        "CM_TOL": 5, #tolerance of flat-bottom binding site restraint, radius in Ang
        "POSRE_FORCE_CONSTANT": 25.0, #force constant of position restraint, in kcal/mol/Ang^2
        "POSRE_TOLERANCE": 3.5, #tolerance of position restraint, radius in Ang
        "ALIGN_KF_SEP": 2.5, #force constant of separation component of the ligand alignment restraint, in kcal/mol/Ang^2
        "ALIGN_K_THETA": 25.0, #force constant of orientation component of the ligand alignment restraint, in kcal/mol
        "ALIGN_K_PSI":   25.0, #force constant of roll component of the ligand alignment restraint, in kcal/mol
        "UMAX": 200.00, #max energy parameter of the soft-core binding energy, in kcal/mol
        "ACORE": 0.062500, #a-parameter of the soft-core binding energy, dimensionless
        "UBCORE": 100.0, #ub-parameter of the soft-core binding energy, in kcal/mol
        "FRICTION_COEFF": 0.500000, #temperature friction relaxation time, in ps
        "TIME_STEP": 0.004, #MD time-step in ps
        "VERBOSE": False,

  }

#job basename
options['BASENAME'] = basename

#displacement
options['DISPLACEMENT'] = Displacement_vector_in_Angstroms

#alignment atoms
options['ALIGN_LIGAND1_REF_ATOMS'] = [i for i in ligand1_ref_atoms]
options['ALIGN_LIGAND2_REF_ATOMS'] = [i for i in ligand2_ref_atoms]

#system pdb file
pdb = PDBFile(pdboutfile)
topology = pdb.topology
positions = pdb.positions

#LIGAND1_ATOMS (residue L1)
res1 = None
for chain in topology.chains():
  for residue in chain.residues():
    if residue.name == 'L1':
        res1 = residue
        break
ligand1_atoms = get_indexes_from_residue(res1)
options['LIGAND1_ATOMS'] = ligand1_atoms

#LIGAND2_ATOMS (residue L2)
res2 = None
for chain in topology.chains():
  for residue in chain.residues():
    if residue.name == 'L2':
        res2 = residue
        break
ligand2_atoms = get_indexes_from_residue(res2)
options['LIGAND2_ATOMS'] = ligand2_atoms

#cm atom of the ligand is the first reference atom
options['LIGAND1_CM_ATOMS'] = [ ligand1_atoms[ ligand1_ref_atoms[0] ] ]
options['LIGAND2_CM_ATOMS'] = [ ligand2_atoms[ ligand2_ref_atoms[0] ] ]

#CMs of the ligands
lig1cm = cm_from_indexes(topology, positions, options['LIGAND1_CM_ATOMS'] )
lig2cm = cm_from_indexes(topology, positions, options['LIGAND2_CM_ATOMS'] )

#CM atoms of the receptor, all C-alpha atoms
query = 'atom.residue.chain.id == "A" and atom.name == "CA"'
options['RCPT_CM_ATOMS'] = get_indexes_from_query(topology, query)

#offset
cmr = cm_from_indexes(topology, positions, options['RCPT_CM_ATOMS'])
options['LIGOFFSET'] = (lig1cm - cmr)/angstrom

#restrained atoms (same as RCPT_CM_ATOMS)
options['POS_RESTRAINED_ATOMS'] = options['RCPT_CM_ATOMS']

#working directory
options['WORKDIR'] = workDir

print("ATM RBFE SETTINGS:")
print(options)

rbfe_structprep(config_file = None, options = options)




### **Run Relative Binding Free Energy Calculation**

It will take 4 hours, or change `WALL_TIME` option above.

In [ ]:
rbfe_production(config_file = None, options = options)

### **Analyisis: Binding free energy estimate**

Runs UWHAM to get the relative binding free energy estimate.

`mintimeid` is used to discard early data. The first two samples, in this example.

In [ ]:

from atom_openmm.uwham import calculate_uwham

ddG, ddG_std, dgbind1, dgbind2, nsamples = calculate_uwham(
          options['WORKDIR'], options['BASENAME'],
          mintimeid = 2,
          intermd=options['INTERMEDIATE'],
          lambda1=options['LAMBDA1'],
          lambda2=options['LAMBDA2'],
          alpha=options['ALPHA'],
          u0=options['U0'],
          w0=options['W0COEFF']
)
print(ddG, '+/-', ddG_std, "kcal/mol")

### **Download the results as a zip file**

In [ ]:
from google.colab import files
print(basename)
!cd /content && zip -r  {basename}.zip {basename}
filename = f'/content/{basename}.zip'
files.download(filename)

### **Optionally, automatically disconnect the runtime when done**

In [ ]:
from google.colab import runtime
import time
time.sleep(300)
runtime.unassign()